# Yum or Yuck Butterfly Mimics 2022
## Baseline Model

A simple Resnet50 transfer learning model using the
CIFAR-100 dataset to establish a baseline score for
the Kaggle Community Competition.

In [4]:
import datetime;

import keras.utils.vis_utils
import tensorflow.python.ops.gen_math_ops

print("executed",datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),"local time")

executed 2022-06-14 20:08:47 local time


---

![Butterfly Classification Diagram](DocResources/ButterflyClassificationTransferLearning-854.png)

Using Tensorflow we will use the weights of the pre-trained Resnet50 convolutional neural network and
replace the last classification layer with our own to classify an image of a butterfly.

In this example we will:

 - Load a dataset using TensorFlow CIFAR-100 Dataset
 - Build a CNN (Convolutional Neural Network)
 - Use TensorFlow 2 Fit, Evaluate, and Predict operations
 - Show a sample of the results
 - Quantify the accuracy of the results

<br>

---

---
## Set Hyperparameters
---

In [ ]:
# Hyper-parameters
BATCH_SIZE = 32
LEARNING_RATE = 0.0003
NUMBER_OF_EPOCHS = 50

IMAGE_SIZE_H = IMAGE_SIZE_V = [224]
MODEL_NAME = 'yoymimics'

SEED = 43

##
---
## Set Environment
---


In [ ]:
import os
import platform
import random

import tensorflow as tf

# Using TensorFlow's enhanced version of Numpy
import tensorflow.experimental.numpy as np
np.experimental_enable_numpy_behavior()

import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import cv2

import tensorflow_datasets as tfds
from keras.preprocessing.image import ImageDataGenerator,load_img
from keras.utils.np_utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Conv2D, \
     MaxPooling2D,Dropout,Flatten,Dense,Activation,BatchNormalization

from tensorflow_datasets.core.registered import DatasetNotFoundError

os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed = SEED
np.random.seed = SEED
tf.random.set_seed(SEED)

print(f"Tensorflow {tf.__version__}", "with GPU support" if len(tf.config.list_physical_devices('GPU')) > 0 else "for CPU only")

##
---
## Load Dataset
---

Our purpose here is to set a baseline score for classifying the
images from the Butterfly Mimics 2022 Dataset.

![Images of Black, Monarch, Pipevine, Spicebush, Tiger and Viceroy butterflies from the dataset](DocResources/the-butterflies.png)

The dataset consists of JPG images of an individual butterfly
and in the training data, a list of labels that identifies the
class of each butterfly as either black, monarch, pipevine,
spicebush, tiger, or viceroy. The classifier will of course need
to predict the label of the image.

![Tiger Swallowtail Butterfly](DocResources/tiger_female_dark_form_vyaa1ee082.jpg)

```python
X = vyaa1ee082.jpg # Features tensor
y = "tiger"        # Target vector
```

We will split the dataset into training and validation data. To work with
the splits we will set the following variables:

 - training split
 - validation split
 - xy_index `{'X': 0, 'y': 1}`
 - supervised_keys `['image', 'label']`
 - class_to_string `{0: 'black', 1: 'monarch', 2: 'pipevine', 3: 'spicebush', 4: 'tiger', 5: 'viceroy'}`
 - number_of_channels


### <u>The train/validate splits</u>

